<a href="https://colab.research.google.com/github/galynz/causal_inference_project/blob/master/train_albert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import pkg_resources
import seaborn as sns
import time
import scipy.stats as stats
import gc
import re
import operator 
import sys
from sklearn import metrics
from sklearn import model_selection
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from nltk.stem import PorterStemmer
from sklearn.metrics import roc_auc_score
%load_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
# import apex
# from apex import amp
import shutil
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AlbertForSequenceClassification

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: ignored

In [0]:
# !pip install torch_xla_py

ERROR: Could not find a version that satisfies the requirement torch_xla_py (from versions: none)
ERROR: No matching distribution found for torch_xla_py


In [0]:
!pip install transformers

In [0]:
!pip uninstall apex

In [0]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setup.sh


In [0]:
!sh setup.sh

In [0]:
import os
import pprint
import tensorflow as tf
if 'COLAB_TPU_ADDR' not in os.environ:
 print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
 tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
 print ('TPU address is', tpu_address)
with tf.Session(tpu_address) as session:
 devices = session.list_devices()
 
 print('TPU devices:')
 pprint.pprint(devices)

ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!


NameError: ignored

In [0]:
torch.cuda.is_available()

True

In [0]:
device=torch.device('cuda')

In [0]:
MAX_SEQUENCE_LENGTH = 220
SEED = 1234
EPOCHS = 1
MODEL_NAME = "albert-base-v1"

from google.colab import drive
drive.mount('/content/drive')
input_dir = "/content/drive/My Drive/Research/abusive_dataset"
features_df = pd.read_csv(f"{input_dir}/hatespeech_features.csv", 
                          lineterminator='\n', index_col=0)
labels_df = pd.read_csv(f"{input_dir}/hatespeech_labels.csv", 
                        index_col=0, dtype={"label": "category"})

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
features_df.shape

(99799, 26)

In [0]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=None,do_lower_case=True)

In [0]:
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

In [0]:
from sklearn.model_selection import train_test_split

founta_sequences = convert_lines(features_df["text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)



0


In [0]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(labels_df.loc[features_df.index, "label"].cat.codes))

Using TensorFlow backend.


In [0]:
Xtrain, Xtest, ytrain, ytest = train_test_split(founta_sequences, 
                                                labels)

In [0]:
train_dataset = torch.utils.data.TensorDataset(torch.tensor(Xtrain,dtype=torch.long), torch.tensor(ytrain,dtype=torch.float))

In [0]:
lr=2e-5
batch_size = 4
accumulation_steps=2
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

model = AlbertForSequenceClassification.from_pretrained(MODEL_NAME,
                                                      cache_dir=None,
                                                      num_labels=4)
model.zero_grad()
model = model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
train = train_dataset

num_train_optimization_steps = int(EPOCHS*len(train)/batch_size/accumulation_steps)

In [0]:
optimizer = AdamW(optimizer_grouped_parameters,
                     lr=lr,
#                      warmup=0.05,
#                      t_total=num_train_optimization_steps
                 )

# model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
model = model.train()

tq = tqdm_notebook(range(EPOCHS))
for epoch in tq:
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    avg_loss = 0.
    avg_accuracy = 0.
    lossf=None
    tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
    optimizer.zero_grad()   # Bug fix - thanks to @chinhuic
    for i,(x_batch, y_batch) in tk0:
#        optimizer.zero_grad()
        y_pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
        loss =  F.binary_cross_entropy_with_logits(y_pred[0],y_batch.to(device))
#         with amp.scale_loss(loss, optimizer) as scaled_loss:
#             scaled_loss.backward()
        loss.backward()
        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            optimizer.step()                            # Now we can do an optimizer step
            optimizer.zero_grad()
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            lossf = loss.item()
        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[0][:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
    tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)

In [3]:
labels_df["label"].value_counts()

NameError: ignored

In [0]:
torch.cuda.empty_cache()